##### *IMP-PCBA*

# Self-study coding activity: Web crawling in action

## Overview
In this activity, you'll compare the use of Beautiful Soup and breadth-first search for web crawling. Use this notebook to follow along with Video 8.4.

This activity is designed to build your familiarity and comfort coding in Python while also helping you review key topics from each module. As you progress through the activity, questions will get increasingly more complex. It is important that you adopt a programmer's mindset when completing this activity. Remember to run your code from each cell before submitting your activity, as doing so will give you a chance to fix any errors before submitting.

### Learning outcome addressed
- Compare the function of Beautiful Soup versus breadth-first search.


In this module, you learned about the Breadth First Search (BFS) algorithm and how it can be used for scraping web pages. The code below is the generic implementation of BFS:

<code>
    def get_children(node):
    # this is the crux of the algo -- you'll need to implement it! 
        return 

    # the BFS algo
    def breadth_first_search(root):
        """
        Returns a list of all visited nodes
        starting from a root node, or seed.
        """
        visited = []
        to_visit = [root]
        while len(to_visit) > 0:
            node = to_visit.pop(0) # poor man's implementation of a queue
            if node not in visited:
                to_visit.extend(get_children(node))
                visited.append(node)
        return visited
</code>

As mentioned in Video 8.4, the implementation of the **get_children** is very domain specific. For this exercise, the **get_children** refers to the hyperlinks contained within a string with html text.

In the second part of this exercise, you will use BeautifulSoup, a Python library that makes it easier to parse and extract data from html pages.


## Index:

- [Question 1](#Question-1)
- [Question 2](#Question-2)
- [Question 3](#Question-3)
- [Question 4](#Question-4)
- [Question 5](#Question-5)

You will be working with the html from the [XKCD](https://xkcd.com/) comic strip. 

Run the cell below to initialize a variable called **xkcd_excerpt** containing some html text from the XKCD web site.

In [1]:
xkcd_excerpt = """<!DOCTYPE html>
                <html>
                <head>
                <link rel="stylesheet" type="text/css" href="/s/b0dcca.css" title="Default"/>
                <title>xkcd: Python</title>
                <meta http-equiv="X-UA-Compatible" content="IE=edge"/>
                <link rel="shortcut icon" href="/s/919f27.ico" type="image/x-icon"/>
                <link rel="icon" href="/s/919f27.ico" type="image/x-icon"/>
                <link rel="alternate" type="application/atom+xml" title="Atom 1.0" href="/atom.xml"/>
                <link rel="alternate" type="application/rss+xml" title="RSS 2.0" href="/rss.xml"/>
                <script type="text/javascript" src="/s/b66ed7.js" async></script>
                <script type="text/javascript" src="/s/1b9456.js" async></script>

                </head>
                <body>
                <div id="topContainer">
                <div id="topLeft">
                <ul>
                <li><a href="/archive">Archive</a></li>
                <li><a href="http://what-if.xkcd.com">What If?</a></li>
                <li><a href="http://blag.xkcd.com">Blag</a></li>
                <li><a href="http://store.xkcd.com/">Store</a></li>
                <li><a rel="author" href="/about">About</a></li>
                </ul>
                </div>
                <div id="topRight">
                <div id="masthead">
                <span><a href="/"><img src="/s/0b7742.png" alt="xkcd.com logo" height="83" width="185"/></a></span>
                <span id="slogan">A webcomic of romance,<br/> sarcasm, math, and language.</span>"""

###### [Back to top](#Index:) 

### Question 1

Let's start by implementing a method called **get_next_url**.  This method takes a page (a string containing html text) and
- Finds the index of the start of the **<a href=**  (start_link).
- Finds the index of the first quote,  starting with the index of **start_link**  (start_quote).
- Finds the index of the end quote, starting with the index of **start_quote** (end_quote).
                                           
Finally, the function returs the string that is between **start_quote** and **end_quote**.  It also returns the index of last quote (**end_quote**).
                                                                                       
**Hint**: Look up the documentation of Python string [find()](https://www.programiz.com/python-programming/methods/string/find).  

In [2]:
def get_next_url(page):
    start_link = page.find('<a href=') 
    start_quote = page.find('"', start_link)  
    end_quote = page.find('"',start_quote + 1) 
    url = page[start_quote + 1: end_quote]
    return url, end_quote


In [3]:
# Test your get_next_url function:
get_next_url(xkcd_excerpt)

('/archive', 978)

## Understanding the get_next_url function

Your function should return:

<code>
    ('/archive', 978)
</code>

As you can see, the first hyperlink the function finds in the text is:

    <a href="/archive">Archive</a>

It returns the value contained within the double quotes (**/archive**) and the index of the last quote (978).  

You will use the index of the last quote to decide where to start looking for your next hyperlink in the page.

###### [Back to top](#Index:) 

### Question 2

The next step is to define the function **get_all_urls**.  This function takes a string as a parameter containing an html text and returns a list of the hyperlinks found in the page.

get_all_urls calls the function you just created called get_next_url.

The idea is to loop inside function get_all_urls calling **get_next_url** until get_next_url does not find any more urls in the page.

Also, note that you will use **end_quote** returned by the **get_next_url** function to slice the array containing the text. That way, you are always working with the next piece of the html text.

In [4]:
def get_all_urls(page):
    """
    this returns all the urls in the html source code 
    return a list with all the url strings
    """
    url_list = [] # list()
    while True:
        url, end_quote = get_next_url(page)
        if url:
            url_list.append(url)
            page = page[end_quote:]
        else:
            break
    return url_list

In [5]:
# Test your get_all_urls function:
get_all_urls(xkcd_excerpt)

['/archive',
 'http://what-if.xkcd.com',
 'http://blag.xkcd.com',
 'http://store.xkcd.com/',
 '/']

## Understanding the get_all_urls function

Your function should return:

<code>
    ['/archive',
     'http://what-if.xkcd.com',
     'http://blag.xkcd.com',
     'http://store.xkcd.com/',
     '/']
</code>

Notice that inside the loop, the **page** string is reset to start where get_next_url last found a hyperlink (end_quote). The loop will stop when get_next_url does not find any more hyperlinks.

###### [Back to top](#Index:) 

### Question 3

Now let's try to implement the BFS algorithm, and let's use a URL instead of a string.  Complete the definition of the function **get_children**.  It takes a URL containing a web page as a parameter and gets the html text for the web page as a string from the **requests** object. Then, it calls the function **get_all_urls**.

The function **get_children** returns an array with all the hyperlinks contained in the page.

In [6]:
import requests

def get_children(url):
    try:
        page_source = requests.get(url, timeout=1).text
    except Exception:
        page_source = ''
    url_list = get_all_urls(page_source)
    return url_list 

In [7]:
## Let's test the function:
get_children('http://www.xkcd.com')

['/archive',
 'https://what-if.xkcd.com',
 '/atom.xml',
 '/newsletter/',
 'https://twitter.com/xkcd/',
 'https://www.facebook.com/TheXKCD/',
 'https://www.instagram.com/xkcd/',
 '/books/',
 '/what-if-2/',
 '/what-if/',
 '/thing-explainer/',
 '/how-to/',
 '/',
 'https://xkcd.com/what-if/',
 'https://bit.ly/WhatIf10th',
 '/1/',
 '//c.xkcd.com/random/comic/',
 '/',
 '/1/',
 '//c.xkcd.com/random/comic/',
 '/',
 'https://xkcd.com/3171',
 'https://imgs.xkcd.com/comics/geologic_core_sample.png',
 '//xkcd.com/1732/',
 '/rss.xml',
 '/atom.xml',
 '/newsletter/',
 'http://threewordphrase.com/',
 'https://www.smbc-comics.com/',
 'https://www.qwantz.com',
 'https://oglaf.com/',
 'https://www.asofterworld.com',
 'https://buttersafe.com/',
 'https://pbfcomics.com/',
 'https://questionablecontent.net/',
 'http://www.buttercupfestival.com/',
 'https://www.homestuck.com/',
 'https://www.jspowerhour.com/',
 'https://medium.com/civic-tech-thoughts-from-joshdata/so-you-want-to-reform-democracy-7f3b1ef10597

The function will return a list with the hyperlinks in the page.  The output should look like the following: 
<code>
    
['/archive',
 'https://what-if.xkcd.com',
 '/atom.xml',
 '/newsletter/',
 'https://twitter.com/xkcd/',
 'https://www.facebook.com/TheXKCD/',
 'https://www.instagram.com/xkcd/',
 '/books/',
 '/what-if-2/',
 '/what-if/',
 '/thing-explainer/',
 '/how-to/',
 '/',
 '/1/',
 '//c.xkcd.com/random/comic/',
 '/',
 '/1/',
 '//c.xkcd.com/random/comic/',
 '/',
 'https://xkcd.com/2757',
 'https://imgs.xkcd.com/comics/towed_message.png',
 '//xkcd.com/1732/',
 '/rss.xml',
 '/atom.xml',
 '/newsletter/',
 'http://threewordphrase.com/',
 'https://www.smbc-comics.com/',
 'https://www.qwantz.com',
 'https://oglaf.com/',
 'https://www.asofterworld.com',
 'https://buttersafe.com/',
 'https://pbfcomics.com/',
 'https://questionablecontent.net/',
 'http://www.buttercupfestival.com/',
 'https://www.homestuck.com/',
 'https://www.jspowerhour.com/',
 'https://medium.com/civic-tech-thoughts-from-joshdata/so-you-want-to-reform-democracy-7f3b1ef10597',
 'https://www.nytimes.com/interactive/2017/climate/what-is-climate-change.html',
 'https://twitter.com/KHayhoe',
 'https://creativecommons.org/licenses/by-nc/2.5/']
 </code>
 
 

###### [Back to top](#Index:) 

### Question 4

Now you are ready to put it all together and implement the BFS algorithm for the web crawler.

Define a function called "**crawl_web** that takes the start_url and the max_depth.

The function returns a dictionary containing key/value pairs with the URL name and the list of hyperlinks for the URL.

In [8]:
def crawl_web(start_url, max_depth):
    """
    Returns a list of all visited nodes
    starting from a root node, or seed.
    """
    crawled = []
    hyperlinksDict = {}
    to_crawl = [[start_url]]
    while to_crawl:
        path = to_crawl.pop(0)
        if len(path) > max_depth:
            break
        url = path[-1]
        if url not in hyperlinksDict:
            children = get_children(url)
            hyperlinksDict[url] = children
            to_crawl.extend([path + [child] for child in children])
    return hyperlinksDict

In [9]:
# test your crawl_web function - using max_depth=1
xkcd_websites = crawl_web(start_url='http://www.xkcd.com', max_depth=1)
xkcd_websites

{'http://www.xkcd.com': ['/archive',
  'https://what-if.xkcd.com',
  '/atom.xml',
  '/newsletter/',
  'https://twitter.com/xkcd/',
  'https://www.facebook.com/TheXKCD/',
  'https://www.instagram.com/xkcd/',
  '/books/',
  '/what-if-2/',
  '/what-if/',
  '/thing-explainer/',
  '/how-to/',
  '/',
  'https://xkcd.com/what-if/',
  'https://bit.ly/WhatIf10th',
  '/1/',
  '//c.xkcd.com/random/comic/',
  '/',
  '/1/',
  '//c.xkcd.com/random/comic/',
  '/',
  'https://xkcd.com/3171',
  'https://imgs.xkcd.com/comics/geologic_core_sample.png',
  '//xkcd.com/1732/',
  '/rss.xml',
  '/atom.xml',
  '/newsletter/',
  'http://threewordphrase.com/',
  'https://www.smbc-comics.com/',
  'https://www.qwantz.com',
  'https://oglaf.com/',
  'https://www.asofterworld.com',
  'https://buttersafe.com/',
  'https://pbfcomics.com/',
  'https://questionablecontent.net/',
  'http://www.buttercupfestival.com/',
  'https://www.homestuck.com/',
  'https://www.jspowerhour.com/',
  'https://medium.com/civic-tech-thoug

In [27]:
# test your crawl_web function - using max_depth=2
xkcd_websites = crawl_web(start_url='http://www.xkcd.com', max_depth=2)
xkcd_websites

{'http://www.xkcd.com': ['/archive',
  'https://what-if.xkcd.com',
  '/atom.xml',
  '/newsletter/',
  'https://twitter.com/xkcd/',
  'https://www.facebook.com/TheXKCD/',
  'https://www.instagram.com/xkcd/',
  '/books/',
  '/what-if-2/',
  '/what-if/',
  '/thing-explainer/',
  '/how-to/',
  '/',
  'https://xkcd.com/what-if/',
  'https://bit.ly/WhatIf10th',
  '/1/',
  '//c.xkcd.com/random/comic/',
  '/',
  '/1/',
  '//c.xkcd.com/random/comic/',
  '/',
  'https://xkcd.com/3171',
  'https://imgs.xkcd.com/comics/geologic_core_sample.png',
  '//xkcd.com/1732/',
  '/rss.xml',
  '/atom.xml',
  '/newsletter/',
  'http://threewordphrase.com/',
  'https://www.smbc-comics.com/',
  'https://www.qwantz.com',
  'https://oglaf.com/',
  'https://www.asofterworld.com',
  'https://buttersafe.com/',
  'https://pbfcomics.com/',
  'https://questionablecontent.net/',
  'http://www.buttercupfestival.com/',
  'https://www.homestuck.com/',
  'https://www.jspowerhour.com/',
  'https://medium.com/civic-tech-thoug

In [ ]:
# test your crawl_web function - using max_depth=3
xkcd_websites = crawl_web(start_url='http://www.xkcd.com', max_depth=3)
xkcd_websites

In [10]:
# how about testing it with a different url:
start_url='https://www.imperial.ac.uk/people/f.nagle'
websites = crawl_web(start_url=start_url, max_depth=2)
websites

{'https://www.imperial.ac.uk/people/f.nagle': ['<!DOCTYPE html>\n\t<!-- image: web-v6.22.0.4 -->\n\t<!-- hash: d2b7ad2cf4da060246348b2c9a31b40531565957 -->\n    <!-- Inter font is copyright (c) 2016 The Inter Project Authors (https://github.com/rsms/inter), included under the SIL license: https://openfontlicense.org/documents/OFL.txt. -->\n    <!-- Barlow Condensed font is copyright 2017 The Barlow Project Authors (https://github.com/jpt/barlow), included under the SIL license: https://openfontlicense.org/documents/OFL.txt. -->\n    <!-- Poppins font is copyright 2014-2019 Indian Type Foundry (info@indiantypefoundry.com), included under the SIL license: https://openfontlicense.org/documents/OFL.txt. -->\n    <!-- Montserrat font is copyright 2011 The Montserrat Project Authors (https://github.com/JulietaUla/Montserrat), included under the SIL license: https://openfontlicense.org/documents/OFL.txt. -->\n\t\n\t<!-- blue -->\n\t<html lang='],
 '<!DOCTYPE html>\n\t<!-- image: web-v6.22.0.4

Note that you may overwhelm your machine if you use max_depth > 3.

###### [Back to top](#Index:) 

### Question 5





In this part of the exercise, you will be using BeautifulSoup to parse the html text.

Use BeautifulSoup to parse the html text in **xkcd_excerpt**


In [22]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(xkcd_excerpt)

print(soup.prettify())

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
<!DOCTYPE html>
<html>
 <head>
  <link href="/s/b0dcca.css" rel="stylesheet" title="Default" type="text/css"/>
  <title>
   xkcd: Python
  </title>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="/s/919f27.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="/s/919f27.ico" rel="icon" type="image/x-icon"/>
  <link href="/atom.xml" rel="alternate" title="Atom 1.0" type="application/atom+xml"/>
  <link href="/rss.xml" rel="alternate" title="RSS 2.0" type="application/rss+xml"/>
  <script async="" src="/s/b66ed7.js" type="text/javascript">
  </script>
  <script async="" src="/s/1b9456.js" type="text/javascript">
  </script>
 </head>
 <body>
  <div id="topContainer">
   <div id="topLeft">
    <ul>
     <li>
      <a href="/archive">
       Archive
      </a>
     </li>
     <li>
      <a href="http://what-if.

###### [Back to top](#Index:) 

### Question 6

Using BeautifulSoup, find the **title** tag in the html text.

In [23]:
hdr =  soup.find("title")
print(hdr)

<title>xkcd: Python</title>


###### [Back to top](#Index:) 

### Question 7

Use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/ ) to parse the data in **xkcd_excerpt** and extract the hyperlinks.

In [24]:
for link in soup.find_all('a', href=True):
    print(link['href'])

/archive
http://what-if.xkcd.com
http://blag.xkcd.com
http://store.xkcd.com/
/about
/


The output should look like the following:
<code>
    /archive
    http://what-if.xkcd.com
    http://blag.xkcd.com
    http://store.xkcd.com/
    /about
    /
</code>

As you can see, BeatifulSoup makes it much easier to parse through html text and extract data.

###### [Back to top](#Index:) 

### Question 7

Retrieve the web page from   [XKCD](https://xkcd.com/), and then use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) to find all hyperlinks contained in the page.

In [25]:
import requests

page_source = requests.get("https://xkcd.com/", timeout=1).text
soup = BeautifulSoup(page_source)

for link in soup.find_all('a', href=True):
    print(link['href'])

/archive
https://what-if.xkcd.com
/about
/atom.xml
/newsletter/
https://twitter.com/xkcd/
https://www.facebook.com/TheXKCD/
https://www.instagram.com/xkcd/
/books/
/what-if-2/
/what-if/
/thing-explainer/
/how-to/
/
https://xkcd.com/what-if/
https://bit.ly/WhatIf10th
/1/
/3170/
//c.xkcd.com/random/comic/
#
/
/1/
/3170/
//c.xkcd.com/random/comic/
#
/
https://xkcd.com/3171
https://imgs.xkcd.com/comics/geologic_core_sample.png
//xkcd.com/1732/
/rss.xml
/atom.xml
/newsletter/
http://threewordphrase.com/
https://www.smbc-comics.com/
https://www.qwantz.com
https://oglaf.com/
https://www.asofterworld.com
https://buttersafe.com/
https://pbfcomics.com/
https://questionablecontent.net/
http://www.buttercupfestival.com/
https://www.homestuck.com/
https://www.jspowerhour.com/
https://medium.com/civic-tech-thoughts-from-joshdata/so-you-want-to-reform-democracy-7f3b1ef10597
https://www.nytimes.com/interactive/2017/climate/what-is-climate-change.html
https://twitter.com/KHayhoe
https://creativecommons

Congratulations! You completed this activity in which you implemented a BFS search algorithm for web crawling and also used BeaitufulSoup to parse and extract data from html text.